In [7]:
import pandas as pd
import numpy as np

In [8]:
file_k = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\kunden.csv"
file_b = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\besucher.csv"
file_geo = r"D:\ryano\OneDrive\Documents\Abschlussprojekt_Python\geo.txt"

In [13]:
df_k = pd.read_csv(file_k, sep = ";", decimal = ".")
df_b = pd.read_csv(file_b, sep = ";", decimal = ",")
df_geo = pd.read_table("geo.txt", delimiter = "\t") #\t muss verwendet werden damit die Spalten der Liste aus der txt Datei separiert werden können

In [14]:
df_geo

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern
...,...,...
1990,K1991,Hessen
1991,K1992,Baden-Württemberg
1992,K1993,Nordrhein-Westfalen
1993,K1994,Berlin


In [16]:
df_b_k = pd.concat([df_b,df_k], ignore_index = True)
df_b_k

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis
0,30,39919.095335,0.0,29.0,K1814,NaN
1,41,53430.673086,0.0,27.0,K1544,NaN
2,64,73163.216927,0.0,32.0,K1945,NaN
3,41,36761.482581,0.0,44.0,K1825,NaN
4,35,42684.331632,1.0,33.0,K1144,NaN
...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58
1991,38,48301.000000,1.0,33.0,K0122,64230.98
1992,67,63547.000000,1.0,41.0,K1045,103123.57
1993,37,32686.000000,1.0,36.0,K1096,49215.87


In [17]:
df_bkg = pd.merge(df_b_k, df_geo, on = "KundeNr")
df_bkg

,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
0,30,39919.095335,0.0,29.0,K1814,NaN,Bayern
1,41,53430.673086,0.0,27.0,K1544,NaN,Baden-Württemberg
2,64,73163.216927,0.0,32.0,K1945,NaN,Baden-Württemberg
3,41,36761.482581,0.0,44.0,K1825,NaN,Baden-Württemberg
4,35,42684.331632,1.0,33.0,K1144,NaN,Nordrhein-Westfalen
...,...,...,...,...,...,...,...
1990,70,69581.000000,1.0,51.0,K0467,103751.58,Thüringen
1991,38,48301.000000,1.0,33.0,K0122,64230.98,Niedersachsen
1992,67,63547.000000,1.0,41.0,K1045,103123.57,Berlin
1993,37,32686.000000,1.0,36.0,K1096,49215.87,Baden-Württemberg
